- https://www.cnblogs.com/bytehandler/p/17635933.html

- `export NCCL_P2P_DISABLE=1`：disable direct GPU-to-GPU (P2P) communication.
    - 禁用点对点通信，点对点通信是一种高效的数据传输方式，允许直接在 GPU 之间传输数据，绕过主机内存（CPU 内存）。但在某些情况下，点对点通信可能会导致兼容性或性能问题。

## `torch.distributed.barrier()`

- 同步进程组内的所有进程，阻塞所有进程直到所有进程都执行到操作。

## 集合通信

NCCL支持集合通信操作（Collective Operations）：

- `AllReduce`，进程组内所有进程进行规约操作，最终所有进程得到统一的Tensor。
- `ReduceScatter`，进程组内所有进程先进行reduce操作，再进行scatter操作，每个进程得到Tensor的一部分。
- `AllGather`，进程组内所有进程的Tensor聚合成一个Tensor列表，并且最终所有进程都有一个Tensor列表副本。
- `Broadcast`，进程组内的一个进程将Tensor广播给其他进程。